In [1]:
import os

In [2]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\English-to-Hindi-Translator\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\English-to-Hindi-Translator'

In [7]:
'''batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1'''

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    batch_size: int
    learning_rate: float
    weight_decay: float
    num_train_epochs: int

In [7]:
from src.translator.constants import *
from src.translator.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            batch_size = params.batch_size,
            learning_rate = params.learning_rate,
            weight_decay = params.weight_decay,
            num_train_epochs = params.num_train_epochs
        )

        return model_trainer_config

In [9]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset,load_from_disk
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

c:\Users\prati\Desktop\Project\English-to-Hindi-Translator\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        tokenized_datasets=load_from_disk(self.config.data_path)
        
        model = TFAutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")
        generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)
        

        #Since my machine is not very powerful so i am training with test data
        train_dataset = model.prepare_tf_dataset(
            tokenized_datasets["test"],
            batch_size=self.config.batch_size,
            shuffle=True,collate_fn=data_collator)
        

        validation_dataset = model.prepare_tf_dataset(
        tokenized_datasets["validation"],
        batch_size=self.config.batch_size,
        shuffle=False,
        collate_fn=data_collator,)

        generation_dataset = model.prepare_tf_dataset(
        tokenized_datasets["validation"],
        batch_size=8,
        shuffle=False,
        collate_fn=generation_data_collator,
)
      
        optimizer = AdamWeightDecay(learning_rate=self.config.learning_rate, weight_decay_rate=self.config.weight_decay)
        model.compile(optimizer=optimizer)
        model.fit(train_dataset, validation_data=validation_dataset, epochs=1)
        

        model.save_pretrained(self.config.root_dir)


In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-08-22 07:01:43,801: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-22 07:01:43,806: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-22 07:01:43,810: INFO: common: created directory at: artifacts]
[2023-08-22 07:01:43,813: INFO: common: created directory at: artifacts/model_trainer]


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.
c:\Users\prati\Desktop\Project\English-to-Hindi-Translator\venv\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


  2/156 [..............................] - ETA: 30:16 - loss: 4.3098  

KeyboardInterrupt: 